In [25]:
#import necessary libraries
import torch
from torch import nn
from tests_backpropagation import main_test

torch.manual_seed(123)
torch.set_default_dtype(torch.double)

## Class ``MyNet``

Read carefully how ``MyNet`` is implemented in the cell below. In particular:  
- ``n_hid`` is a list of integer, representing the number of hidden units in each hidden layer.   
-  ``MyNet([2, 3, 2]) = MiniNet()`` where ``MiniNet`` is the neural network defined in the fourth tutorial, in which notations are also clarified.     
- ``model.L`` is the number of hidden layers, ``L``   
- ``model.f[l]`` is the activation function of layer ``l``, $f^{[l]}$ (here ``torch.tanh``)   
- ``model.df[l]`` is the derivative of the activation function, $f'^{[l]}$   
- ``model.a[l]``  is the tensor $A^{[l]}$, (shape: ``(1, n(l))``)   
- ``model.z[l]``  is the tensor $Z^{[l]}$, (shape: ``(1, n(l))``)  
- Weights $W^{[l]}$ (shape: ``(n(l+1), n(l))``) and biases $\mathbf{b}^{[l]}$ (shape: ``(n(l+1))``) can be accessed as follows:
```
weights = model.fc[str(l)].weight.data
bias = model.fc[str(l)].bias.data
```

In [26]:
class MyNet(nn.Module):
    def __init__(self, n_l = [2, 3, 2]):
        super().__init__() 
        
        
        # number of layers in our network (following Andrew's notations)
        self.L = len(n_l)-1
        self.n_l = n_l
        
        # Where we will store our neuron values
        # - z: before activation function 
        # - a: after activation function (a=f(z))
        self.z = {i : None for i in range(1, self.L+1)}
        self.a = {i : None for i in range(self.L+1)}

        # Where we will store the gradients for our custom backpropagation algo
        self.dL_dw = {i : None for i in range(1, self.L+1)}
        self.dL_db = {i : None for i in range(1, self.L+1)}

        # Our activation functions
        self.f = {i : lambda x : torch.tanh(x) for i in range(1, self.L+1)}

        # Derivatives of our activation functions
        self.df = {
            i : lambda x : (1 / (torch.cosh(x)**2)) 
            for i in range(1, self.L+1)
        }
        
        # fully connected layers
        # We have to use nn.ModuleDict and to use strings as keys here to 
        # respect pytorch requirements (otherwise, the model does not learn)
        self.fc = nn.ModuleDict({str(i): None for i in range(1, self.L+1)})
        for i in range(1, self.L+1):
            self.fc[str(i)] = nn.Linear(in_features=n_l[i-1], out_features=n_l[i])
        
    def forward(self, x):
        # Input layer
        self.a[0] = torch.flatten(x, 1)
        
        # Hidden layers until output layer
        for i in range(1, self.L+1):

            # fully connected layer
            self.z[i] = self.fc[str(i)](self.a[i-1])
            # activation
            self.a[i] = self.f[i](self.z[i])

        # return output
        return self.a[self.L]

## Tasks

Write a function ``backpropagation(model, y_true, y_pred)`` that computes:

- $\frac{\partial L}{\partial w^{[l]}_{i,j}}$ and store them in ``model.dL_dw[l][i,j]`` for $l \in [1 .. L]$ 
- $\frac{\partial L}{\partial b^{[l]}_{j}}$ and store them in ``model.dL_db[l][j]`` for $l \in [1 .. L]$ 

assuming ``model`` is an instance of the ``MyNet`` class.

A vectorized implementation would be appreciated.

In [27]:
def backpropagation(model, y_true, y_pred):
    """
    Manually compute gradients dL/dw and dL/db for each layer in MyNet.
    Assumes batch_size = 1, so shapes are mostly (1, n[l]).
    """
    with torch.no_grad():
        
        L = model.L
        
        #------------------------------------------------
        # 1) Output-layer delta
        #    delta_curr will stay shape (1, n[L]) if y_pred is also (1, n[L]) 
        #------------------------------------------------
        delta_curr = dloss(y_true, y_pred) * model.df[L](model.z[L])
        # Weight gradients: shape (n[L], n[L-1])
        #   delta_curr.view(-1,1) is (n[L],1)
        #   model.a[L-1] is (1,n[L-1]) => broadcast => (n[L], n[L-1])
        model.dL_dw[L] = delta_curr.view(-1, 1) * model.a[L-1]
        
        # Bias gradients: shape (n[L],)
        #   delta_curr is (1,n[L]), so delta_curr.squeeze(0) => (n[L],)
        model.dL_db[L] = delta_curr.squeeze(0)
        
        #------------------------------------------------
        # 2) Backprop for hidden layers
        # move backward from l = L-1 down to l=1
        #------------------------------------------------
        for l in range(L-1, 0, -1):
            # delta_prev = delta_curr
            weights_prev = model.fc[str(l+1)].weight.data  # shape (n[l+1], n[l])
            
            #   delta_curr: (1, n[l+1]) => delta_curr.view(-1,1): (n[l+1],1)
            #   weights_prev: (n[l+1], n[l]) => elementwise multiply => (n[l+1], n[l])
            tmp = delta_curr.view(-1, 1) * weights_prev
            
            # sum over axis=0 => shape (n[l],). We'll keep dim so it remains (1, n[l])
            tmp = tmp.sum(dim=0, keepdim=True)  # => (1, n[l])
            
            # multiply by derivative of activation => shape (1, n[l])
            delta_curr = tmp * model.df[l](model.z[l])
            
            # Weight gradient => shape (n[l], n[l-1]) 
            model.dL_dw[l] = delta_curr.view(-1, 1) * model.a[l-1]
            
            # Bias gradient => shape (n[l],)
            model.dL_db[l] = delta_curr.squeeze(0)

## Run the cells below, and check the output

- In the 1st cell, we use a toy dataset and the same architecture as the MiniNet class of the fourth tutorial. 
- In the 2nd cell, we use a few samples of the MNIST dataset with a consistent model architecture (``24x24`` black and white cropped images as input and ``10`` output classes). 

You can set ``verbose`` to ``True`` if you want more details about your computations versus what is expected.

In [28]:
model = MyNet([2, 3, 2])
main_test(backpropagation, model, verbose=True, data='toy')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

 --------- Comparing with autograd values  ----------- 

 ******* fc['1'].weight.grad ******* 
  Our computation:
 tensor([[ 6.7814e-08,  7.2355e-08],
        [-1.0720e-03, -1.1438e-03],
        [-2.2284e-03, -2.3776e-03]])

  Autograd's computation:
 tensor([[ 6.7814e-08,  7.2355e-08],
        [-1.0720e-03, -1.1438e-03],
        [-2.2284e-03, -2.3776e-03]])

 ********* fc['1'].bias.grad ******* 
  Our computation:
 tensor([ 8.0769e-09, -1.2768e-04, -2.6541e-04])
  Autograd's computation:
 tensor([ 8.0769e-09, -1.2768e-04, -2.6541e-04])

 ------------------- relative error ------------------ 
(fc[1]

In [29]:
model = MyNet([24*24, 16, 10])
main_test(backpropagation, model, verbose=True, data='mnist')


 __________________________________________________________________ 
                          Check gradients                             
 __________________________________________________________________ 

 ====================== Epoch 1 ====================== 

 -------- Gradcheck with finite differences  --------- 
 residual error:
 [0.0, 0.0, 0.0, 0.0, 0.0]

 --------- Comparing with autograd values  ----------- 

 ******* fc['1'].weight.grad ******* 
  Our computation:
 tensor([[  1.7421e-27,   1.7421e-27,   1.7421e-27,  ...,   1.7421e-27,
           1.7421e-27,   1.7421e-27],
        [-5.9696e-177, -5.9696e-177, -5.9696e-177,  ..., -5.9696e-177,
         -5.9696e-177, -5.9696e-177],
        [ -3.7006e-19,  -3.7006e-19,  -3.7006e-19,  ...,  -3.7006e-19,
          -3.7006e-19,  -3.7006e-19],
        ...,
        [-1.6763e-132, -1.6763e-132, -1.6763e-132,  ..., -1.6763e-132,
         -1.6763e-132, -1.6763e-132],
        [  1.4466e-40,   1.4466e-40,   1.4466e-40,  ...,   1.4466e-